In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from PIL import Image, ImageColor, ImageFont, ImageDraw
import matplotlib.pyplot as plt
import time

import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO
from PIL import ImageOps

In [2]:
def load_images():
  imgs = []
  for img_suffix in range(1, 12):
    img = Image.open('drive/MyDrive/TIES4911/task5/img-{}.jpg'.format(img_suffix))
    img = np.array(img, dtype=np.uint8)
    imgs.append(img)

  return imgs

In [3]:
def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin
    
def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

In [4]:
def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)
  plt.show()

In [6]:
def run_detector(detector, img):
  start_time = time.time()
  result = detector(img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}

  print("Found %d objects." % len(result["detection_scores"]))
  print("Inference time: ", end_time-start_time)

  return result
  

In [11]:
batch = load_images()

In [8]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
detector = hub.load(module_handle).signatures['default']

In [9]:
for img in batch:
  result = run_detector(detector, tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...])

  image_with_boxes = draw_boxes(
        img, result["detection_boxes"],
        result["detection_class_entities"], result["detection_scores"])

  display_image(image_with_boxes)

Output hidden; open in https://colab.research.google.com to view.

In [12]:
module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"
detector = hub.load(module_handle).signatures['default']

for img in batch:
  result = run_detector(detector, tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...])

  image_with_boxes = draw_boxes(
        img, result["detection_boxes"],
        result["detection_class_entities"], result["detection_scores"])

  display_image(image_with_boxes)

Output hidden; open in https://colab.research.google.com to view.

In [13]:
print(result['detection_scores'])

[0.42921713 0.38905618 0.33126694 0.2803415  0.276144   0.2505066
 0.2480646  0.23329891 0.22441083 0.20431413 0.20275888 0.19790405
 0.18916887 0.17215958 0.1552243  0.1551608  0.15185823 0.1498537
 0.14906396 0.14082429 0.14058293 0.13972203 0.13827924 0.13327277
 0.13175508 0.12657513 0.12623976 0.1235878  0.12355895 0.12092101
 0.12068781 0.12028635 0.11865243 0.11822836 0.11739748 0.11698569
 0.11540227 0.11505985 0.11403608 0.11378246 0.10762031 0.10701652
 0.10654689 0.10632709 0.10631355 0.1059767  0.10594515 0.10448881
 0.10396284 0.10342074 0.10338169 0.09985514 0.09981069 0.09956084
 0.09948576 0.09925487 0.09892286 0.09834023 0.09820155 0.09789157
 0.09762847 0.09754497 0.09723479 0.09717809 0.09710706 0.09699024
 0.09697199 0.09583829 0.09564037 0.09545739 0.09495333 0.09457339
 0.09371901 0.09368592 0.09356103 0.09334313 0.09274854 0.09248992
 0.09208734 0.08988956 0.08985413 0.08907989 0.08886461 0.08851621
 0.0878135  0.0874358  0.08707461 0.08642296 0.08623574 0.086153